In [71]:
# @ Haosen He
# Wake Forest Univeristy
# RA work for Prof. Tin Cheuk Leung
# May 2020
from selenium import webdriver # A web driver like chromium is required.
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
wordlist="Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bohemia and Moravia,Bolivia,Bonaire,Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,Central African Republic,Chad,Chile,China,CIS,Colombia,Comoros,Congo,Cook Islands,Costa Rica,Côte d Ivoire,Croatia,Cuba,Curacao,Cyprus,Czech Republic,Czechoslovakia,Denmark,Djibouti,Dominica,Dominican Republic,DR Congo,Dutch East Indies,Ecuador,Egypt,El Salvador,England,Equatorial Guinea,Eritrea,Estonia,Ethiopia,Faroe Islands,Fiji,Finland,France,French Guiana,FYR of Macedonia,Gabon,Gambia,Georgia,German Dem Republic,Germany,Ghana,Gibraltar,Great Britain,Greece,Grenada,Guadeloupe,Guam,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithania,Luxembourg,Macau,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Martinique,Mauritania,Mauritius,Mayotte,Mexico,Micronesia,Moldova,Mongolia,Montenegro,Montserrat,Morocco,Mozambique,Myanmar,Namibia,Nauru,Nepal,Netherlands,Netherlands Antilles,New Caledonia,New Zealand,Nicaragua,Niger,Nigeria,Niue,North Korea,North Vietnam,North Yemen,Northern Ireland,Northern Mariana Islands,Norway,Oman,Pakistan,Palau,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Puerto Rico,Qatar,RCS,Réunion,Romania,Russia,Rwanda,Saarland,Saint-Martin,Samoa,San Marino,Sansibar,São Tomé and Príncipe,Saudi Arabia,Scotland,Senegal,Serbia,Serbia and Montenegro,Seychelles,Sierra Leone,Singapore,Sint Maarten,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,South Yemen,Soviet Union,Spain,Sri Lanka,St. Kitts and Nevis,St. Lucia,St. Pierre and Miquelon,St. Vincent and the Grenadines,Sudan,Suisse,Suriname,Swaziland,Sweden,Syria,Tahiti,Taiwan,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Tonga,Trinidad and Tobago,Tunisia,Turkey,Turkmenistan,Turks and Caicos Islands,Tuvalu,Uganda,Ukraine,United Arab Emirates,Uruguay,US Virgin Islands,USA,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wales,Yemen,Yugoslavia,Zambia,Zimbabwe,"
nations=[]
for i in range(len(wordlist)):
    if wordlist[i]==',':
        nations.append(wordlist[temp:i])

In [73]:
driver=webdriver.Chrome()
driver.get("http://laenderspiel.cmuck.de/index.php?tlm=statistik&utlm=Team")       #Open the page
time.sleep(0.1)   
driver.find_element_by_xpath("//img[@ src='grafik/flaggen/gbr.png']").click();
time.sleep(0.1)
driver.find_element_by_id("navStatistik").click()
time.sleep(0.1)
driver.find_element_by_link_text('Single Team').click()
time.sleep(0.1)
driver.find_element_by_id('MonatVon').send_keys('01.01.1800')
driver.find_element_by_id('MonatBis').send_keys('10.05.2020')
time.sleep(0.1)

for n in nations:
    data={"1":[], "2":[], "3":[], "4":[], "5":[], "6":[], "7":[], "8":[], "9":[], "10":[], "11":[]}
    
    driver.find_element_by_id("Nation").send_keys(n)
    time.sleep(0.1)
    driver.find_element_by_id("SpieleSuchen").click()
    time.sleep(10)
    html=driver.page_source
    soup1=BeautifulSoup(html,'lxml')
    stat_table=soup1.find("table", id="ErgebnisResult")

    for tr in stat_table.findAll('tr'):
        count=1
        for td in tr.findAll('td'):
            data[str(count)].append(td.getText().replace('\n', '').replace('\t', ''))
            count+=1
    nationfile=pd.DataFrame(data).drop(columns=['3', '7', '8', '9', '10', '11']) # columns we don't need
    nationfile.to_csv (r'C:\\Users\\s1760\\Desktop\\Soccer\\'+ n +'.csv', index = False, header=True) # raw csv

In [135]:
# organize collected data
for i in nations:
    tempdata=pd.read_csv('C:\\Users\\s1760\\Desktop\\Soccer\\'+ i +'.csv')
    tempdata.rename(columns={"1": "date", "2": "location","4": "team1","5":"result","6":"team2"},inplace=True)
    team1score=[]
    team2score=[]
    winner=[]
    for j in tempdata['result']:
        j=j.replace(' - ', ',')
        templs=j.split(',')
        sc1=int(templs[0])
        sc2=int(templs[1])
        team1score.append(sc1)
        team2score.append(sc2)
        if sc1 > sc2:
            winner.append('team1')
        elif sc1<sc2:
            winner.append('team2')
        else:
            winner.append("NaN")
    tempdata['team1score']=team1score
    tempdata['team2score']=team2score
    win=[]
    for k in range(0, len(winner)):
        if winner[k]=='team1':
            win.append(tempdata['team1'][k])
        elif winner[k]=='team2':
            win.append(tempdata['team2'][k])
        else:
            win.append("NaN")
    tempdata['winner']=win
    tempdata.to_csv (r'C:\\Users\\s1760\\Desktop\\Soccer2\\'+ i +'.csv', index = False, header=True)